In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template(
  "What is the distance between {country_a} and {country_b}",)


prompt = template.format(country_a="France", country_b="Germany")

chat.predict(prompt)

'The distance between France and Germany is approximately 500 kilometers (310 miles) when measured from Paris, France to Berlin, Germany.'

In [7]:
template = ChatPromptTemplate.from_messages(
  [
    ("system", "you are a geography expert. And you only reply in {language}"),
    ("ai", "Ciao, mi chiamo {name}"),
    (
      "human",
      "What is the distance between {country_a} and {country_b}, Also, what is your name?",
    ),
  ]
)


prompt = template.format_messages(
  language="Greek",
  name="Socrates",
  country_a="Mexico",
  country_b="Thailand",
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα.')